Notebook to analyse user data

In [1]:
import yaml
import os
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect

In [2]:
class DatabaseConnector:
    def __init__(self, base_path, cred_file):
        self.base_path = base_path
        self.cred_file = cred_file

    def read_db_creds(self):
        full_file_path = os.path.join(self.base_path, self.cred_file)
        with open(full_file_path, 'r') as file:
            data = yaml.safe_load(file)
            return data

    def init_db_engine(self):
        database_cred = self.read_db_creds()
        database_uri = f"postgresql+psycopg2://{database_cred['RDS_USER']}" \
                       f":{database_cred['RDS_PASSWORD']}@{database_cred['RDS_HOST']}:" \
                       f"{database_cred['RDS_PORT']}/{database_cred['RDS_DATABASE']}"
        database_engine = create_engine(database_uri)
        return database_engine

    def list_db_table(self):
        engine = self.init_db_engine()
        inspector = inspect(engine)
        list_all_tables = inspector.get_table_names()
        return list_all_tables

    def upload_to_db(self, df, table_name):
        engine = self.init_db_engine()
        df.to_sql(name=table_name, con=engine)


In [3]:
class DataExtractor:

    def read_rds_table(self, data_connector, table_name):

        all_tables = data_connector.list_db_table()
        for table in all_tables:
            if table_name == table:
                df = pd.read_sql_table(table, data_connector.init_db_engine())
                return df

    def retrieve_pdf_data(self, pdf_path):
        dfs = tabula.read_pdf(pdf_path, lattice=True, pages="all")
        df = pd.concat(dfs)
        return df

In [7]:
base_path = r"C:\Users\abhik\Documents\aicore\multinational-retail-data-centralisation946"
data_connector = DatabaseConnector(base_path=base_path, cred_file="db_creds.yaml")
data_extractor = DataExtractor()
data_connector.list_db_table()


['legacy_store_details', 'legacy_users', 'orders_table']

In [9]:
data_df = data_extractor.read_rds_table(data_connector, table_name="legacy_users")
data_df.head()

,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579


In [13]:
data_df.shape

(15320, 12)

In [14]:
data_df.isnull().sum()

index            0
first_name       0
last_name        0
date_of_birth    0
company          0
email_address    0
address          0
country          0
country_code     0
phone_number     0
join_date        0
user_uuid        0
dtype: int64

In [15]:
data_df.isna().sum()

index            0
first_name       0
last_name        0
date_of_birth    0
company          0
email_address    0
address          0
country          0
country_code     0
phone_number     0
join_date        0
user_uuid        0
dtype: int64

In [31]:
if pd.to_datetime(data_df['date_of_birth'], format='%Y-%m-%d', errors='coerce').notnull().all():
    print("no error")
else:
    print("wrong date format present")
    

wrong date format present


In [30]:
try:
    pd.to_datetime(data_df['date_of_birth'], format='%Y-%m-%d', errors="raise")
except ValueError as e:
    print(e)

time data "1968 October 16" doesn't match format "%Y-%m-%d", at position 360. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.


In [32]:
nan_df = pd.to_datetime(data_df['date_of_birth'], format='%Y-%m-%d', errors="coerce")
# above will return NaN when the format not match 
clean_df = data_df[nan_df.notna()]

In [33]:
pd.to_datetime(clean_df['date_of_birth'], format='%Y-%m-%d')

0       1990-09-30
1       1940-12-01
2       1995-08-02
3       1972-09-23
4       1952-12-20
           ...    
15315   1943-08-09
15316   1948-08-20
15317   1940-10-09
15318   1952-06-04
15319   1994-03-27
Name: date_of_birth, Length: 15257, dtype: datetime64[ns]

In [34]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15257 entries, 0 to 15319
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          15257 non-null  int64 
 1   first_name     15257 non-null  object
 2   last_name      15257 non-null  object
 3   date_of_birth  15257 non-null  object
 4   company        15257 non-null  object
 5   email_address  15257 non-null  object
 6   address        15257 non-null  object
 7   country        15257 non-null  object
 8   country_code   15257 non-null  object
 9   phone_number   15257 non-null  object
 10  join_date      15257 non-null  object
 11  user_uuid      15257 non-null  object
dtypes: int64(1), object(11)
memory usage: 1.5+ MB


In [44]:

data_df.describe()


,index
count,15320.000000
mean,7659.500000
std,4422.647397
min,0.000000
25%,3829.750000
50%,7659.500000
75%,11489.250000
max,15319.000000
